In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
from sklearn.linear_model import LogisticRegression
import spacy
from tqdm import tqdm

sst_dataset = {}
for split in ["train", "dev", "test"]:
    URL = f"https://raw.githubusercontent.com/successar/instance_attributions_NLP/master/Datasets/SST/data/{split}.jsonl"
    import urllib.request, json
    with urllib.request.urlopen(URL) as url:
        data = url.read().decode()
        data = [json.loads(line) for line in data.strip().split("\n")]
        sst_dataset[split] = data

nlp = spacy.load('en_core_web_md')
X, y = {}, {}
for split in ["train", "dev"]:
    X[split] = np.array([nlp(example["document"]).vector for example in tqdm(sst_dataset[split])])
    y[split] = np.array([example["label"] for example in sst_dataset[split]])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 872/872 [00:03<00:00, 242.88it/s]
/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
%load_ext autoreload
%autoreload 2

import numpy as np
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

sst_dataset = {}
for split in ["train", "dev", "test"]:
    URL = f"https://raw.githubusercontent.com/successar/instance_attributions_NLP/master/Datasets/SST/data/{split}.jsonl"
    import urllib.request, json
    with urllib.request.urlopen(URL) as url:
        data = url.read().decode()
        data = [json.loads(line) for line in data.strip().split("\n")]
        sst_dataset[split] = data

X = {}
y = {}
path = "./SST/"
X["train"] = np.load(path + "train_feature_save.npy",allow_pickle=True )
y["train"] = np.load(path + "train_label_save.npy").squeeze()
X["dev"] = np.load(path + "test_feature_save.npy")
y["dev"] = np.load(path + "test_label_save.npy").squeeze()

print(X["train"].shape, y["train"].shape, X["dev"].shape, y["dev"].shape)
thresh = 0.5
l2 = 500
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(6920, 768) (6920,) (872, 768) (872,)


LogisticRegression(C=0.002)

In [11]:
from MinimalSubsetToFlipPredictions.evaluate import compute_subset_metrics, evaluate_predictions_flip_variant

thresh = 0.5
# l2 = 1
l2 = 10

model = LogisticRegression(penalty='l2', C=1/l2, max_iter=1000)
model.fit(X["train"], y["train"])

# fname = "yang_fast_SST.pkl"
fname = "yang_slow_SST.pkl"
import pickle
with open(fname, "rb") as handle:
    flip_list = pickle.load(handle)

flip_list

## Algo 1 SST BOW
# {'Coverage': 82.0,
#  'Overall Validity': 76.49,
#  'Precision Validity': 93.29,
#  'Median Size': 60.0}

## Algo 2 SST BOW
# {'Coverage': 82.0,
#  'Overall Validity': 73.51,
#  'Precision Validity': 89.65,
#  'Median Size': 52.0}

is_valid = evaluate_predictions_flip_variant(
    clf=model,
    flip_list=flip_list,
    train_embeddings=X['train'],
    train_labels=y['train'],
    test_embeddings=X['dev'],
    ex_indices_to_check=np.arange(y['dev'].size)
)

compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

100%|██████████| 872/872 [07:01<00:00,  2.07it/s]

641/715 identified subsets are valid
Overall validity is 641/872, or 73.51%
Precision validity is 641/715, or 89.65%
Identified 715/872 subsets.
Coverage: 82.0%
Median Valid Subset Sizes is 52.0, out of 641 valid subsets


{'Coverage': 82.0,
 'Overall Validity': 73.51,
 'Precision Validity': 89.65,
 'Median Size': 52.0}

In [5]:
# new_fname = "new_predictions_alg1_SST1.npy"
new_fname = "new_predictions_SST_alg2.npy"

new_pred = np.load(new_fname, allow_pickle=True)

# old_fname = "old_predictions_alg1_SST1.npy"
old_fname = "old_predictions_SST_alg2.npy"

old_pred = np.load(old_fname, allow_pickle=True)

true_old_pred = old_pred >= 0.5

# true_new_pred[:5], 
# true_old_pred[:5]

is_valid_3 = []
for po, pn in zip(true_old_pred, new_pred):
    if pn is None:
        is_valid_3.append(False)
        continue
    pr = pn >= 0.5
    # print(pr, po[0])
    if pr != po[0]:
        is_valid_3.append(True)
    else:
        is_valid_3.append(False)

is_valid_3 = np.array(is_valid_3)
sum(is_valid_3), is_valid_3.size, sum(is_valid_3)/is_valid_3.size

(636, 872, 0.7293577981651376)

In [ ]:
from sklearn import clone
from sklearn.base import BaseEstimator

from utils.models import get_predictions


def retrain_and_evaluate_validity(
    clf: BaseEstimator,
    train_embeddings: np.ndarray,
    train_labels: np.ndarray,
    x_test: np.ndarray,
    indices_to_exclude: np.ndarray,
):
    train_mask = np.ones(train_embeddings.shape[0], dtype=bool)
    train_mask[indices_to_exclude] = False
    reduced_embeddings = train_embeddings[train_mask]
    reduced_labels = train_labels[train_mask]
    old_pred = get_predictions(clf, x_test.reshape(1, -1))[0]
    new_clf = clone(clf)
    new_clf.fit(reduced_embeddings, reduced_labels)
    new_pred = get_predictions(new_clf, x_test.reshape(1, -1))[0]
    # this subset is valid only if new prediction does not equal old prediction
    return old_pred, new_pred, new_pred != old_pred

to_exclude = [6867, 6696, 4180, 4225, 6648, 4519, 5009, 4436, 3559,  882,  398, 4423, 4294]

retrain_and_evaluate_validity(
    clf=model,
    flip_list=flip_list,
    train_embeddings=X['train'],
    train_labels=y['train'],
    test_embeddings=X['dev'],
    ex_indices_to_check=np.arange(y['dev'].size)
)

In [69]:
is_valid_3

array([ True, False,  True, False,  True,  True,  True, False, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True, False, False,  True, False,
        True,  True, False, False,  True, False, False,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False,  True, False, False, False, False, False,  True,
        True,  True, False,  True, False, False,  True, False,  True,
        True, False,  True,  True,  True, False,  True, False, False,
       False,  True, False, False, False, False,  True,  True,  True,
        True,  True,

In [70]:
flip_list[2]

array([6534, 6269, 5201, 4504, 5272, 6118, 5823, 6342, 4424, 5941, 5864,
       1987,  391, 4064, 6013, 3866, 6453, 4922, 6864,  662, 6659, 3461,
       5644, 1361, 4701, 4825,  906, 3667, 1005, 6574, 3889,   63, 1957,
       2127, 1542])

In [74]:
from utils.models import get_predictions
from sklearn.base import clone

indices_to_exclude = [6534, 6269, 5201, 4504, 5272, 6118, 5823, 6342, 4424, 5941, 5864,
       1987,  391, 4064, 6013, 3866, 6453, 4922, 6864,  662, 6659, 3461,
       5644, 1361, 4701, 4825,  906, 3667, 1005, 6574, 3889,   63, 1957,
       2127, 1542]
train_mask = np.ones(X['train'].shape[0], dtype=bool)
train_mask[indices_to_exclude] = False
reduced_embeddings = X['train'][train_mask]
reduced_labels = y['train'][train_mask]

old_pred = get_predictions(model, X['dev'][2].reshape(1, -1))[0]
new_clf = clone(model)
new_clf.fit(reduced_embeddings, reduced_labels)
new_pred = get_predictions(new_clf, X['dev'][2].reshape(1, -1))[0]

print(model, new_clf, old_pred, new_pred)

LogisticRegression(C=0.001) LogisticRegression(C=0.001) 1 1


In [1]:
import pickle 
with open("esnli_deberta_large_yang_fast_0to2_is_valid_subsets.pickle", "rb") as handle:
    d = pickle.load(handle)

d

[False, False]